In [108]:
#imports
#load in standard data 
#load in models

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import pandas as pd
import sklearn
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,classification_report,plot_confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier

pd.set_option('display.max_columns', 500)
import pickle as pkl
vtc1 = pkl.load(open('vtc.pkl','rb'))
vtc2 = pkl.load(open('vtcpoly.pkl','rb'))
df1 = pd.read_csv('../prepeddata.csv',index_col='Unnamed: 0')
df1['stimulant'] = df1['stimulant'].apply(lambda x: 'low' if x <= 2 else 'high')
df1['depressant'] = df1['depressant'].apply(lambda x: 'low' if x <= 2 else 'high')
df1['hallucinagen'] = df1['hallucinagen'].apply(lambda x: 'low' if x <= 2 else 'high')
x = df1.drop(['stimulant','hallucinagen','depressant'],axis = 1)
ystimulants = df1['stimulant']
yhallucinagen =df1['hallucinagen']
ydepressant = df1['depressant']
y=df1[['stimulant','hallucinagen','depressant']]

poly=PolynomialFeatures(degree=3,interaction_only=True)
scaler = StandardScaler()
polyscaler = StandardScaler()
selector = SelectKBest(k=43)

In [109]:
#traintest split default data 
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(x,stimy)
sm1 = SMOTE()
tl1 = TomekLinks()
sampling = SMOTETomek( random_state=None, smote=sm1, tomek=tl1, n_jobs=1)


stimXtrain, stimYtrain = sampling.fit_resample(stimXtrain, stimYtrain)

haluXtrain,haluXtest,haluYtrain,haluYtest = train_test_split(x,yhallucinagen)
sm2 = SMOTE()
tl2 = TomekLinks()
sampling1 = SMOTETomek( random_state=None, smote=sm2, tomek=tl2, n_jobs=1)


haluXtrain, haluYtrain = sampling1.fit_resample(haluXtrain, haluYtrain)

depXtrain,depXtest,depYtrain,depYtest = train_test_split(x,ydepressant)
sm3 = SMOTE()
tl3 = TomekLinks()
sampling2 = SMOTETomek( random_state=None, smote=sm3, tomek=tl3, n_jobs=1)


depXtrain, depYtrain = sampling2.fit_resample(depXtrain, depYtrain)

In [110]:
df1['hallucinagen'].describe()
haluXtrain,depXtrain

(      Age  Education     Nscore     Escore     Oscore     Ascore     Cscore  \
 0       2          3  47.000000  38.000000  52.000000  39.000000  48.000000   
 1       2          4  24.000000  37.000000  44.000000  47.000000  48.000000   
 2       1          3  34.000000  45.000000  50.000000  44.000000  47.000000   
 3       2          4  30.000000  42.000000  45.000000  44.000000  46.000000   
 4       1          3  18.000000  42.000000  51.000000  54.000000  48.000000   
 ...   ...        ...        ...        ...        ...        ...        ...   
 2162    1          2  28.309251  36.762995  43.690749  35.762995  41.618503   
 2163    1          0  29.704274  44.481709  49.295726  54.740855  52.222564   
 2164    2          1  30.635848  32.038814  41.519407  34.961186  48.077627   
 2165    1          1  28.623504  37.752991  41.258974  40.623504  39.623504   
 2166    1          0  44.958581  33.020710  54.041419  43.020710  36.896452   
 
       Impulsive  SensationSeeking  Al

In [111]:
#make a pipeline of standard vtc
stdvtc = Pipeline([('scaler',scaler),('model', vtc1)])
stdvtc.fit(stimXtrain,stimYtrain)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 VotingClassifier(estimators=[('dtc2',
                                               DecisionTreeClassifier(ccp_alpha=0.0052,
                                                                      criterion='entropy',
                                                                      max_depth=50)),
                                              ('rfc2',
                                               RandomForestClassifier(ccp_alpha=0.01,
                                                                      class_weight='balanced',
                                                                      criterion='entropy',
                                                                      n_estimators=10000,
                                                                      oob_score=True)),
                                              ('knn',
                                               KN

In [112]:
#instantiate d3 interaction only polyfeats

In [113]:
#instantiate select k best k =43

In [114]:
#instantiate standardscaler

In [115]:
#make a pipeline of poly vtc in order
# polyfeats
#select k best
#standardscaler
polyvtc = Pipeline([('poly',poly),('selector', selector),('scaler',polyscaler),('model', vtc2)])
polyvtc.fit(stimXtrain,stimYtrain)

C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [0] are constant.
  UserWarning)
C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Pipeline(steps=[('poly', PolynomialFeatures(degree=3, interaction_only=True)),
                ('selector', SelectKBest(k=43)), ('scaler', StandardScaler()),
                ('model',
                 VotingClassifier(estimators=[('polymodel1',
                                               LogisticRegression(C=0.75,
                                                                  class_weight='auto',
                                                                  max_iter=1000,
                                                                  solver='sag',
                                                                  warm_start=True)),
                                              ('polymodel2',
                                               LogisticRegression(C=10,
                                                                  class_weight='auto',
                                                                  fit_intercept=...
                                            

In [116]:
# fit both

In [117]:
#check classification report for both
vtcypred = stdvtc.predict(stimXtest)
vtcytrain = stdvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

0.8401759215877274 0.7926234492399091
              precision    recall  f1-score   support

        high       0.79      0.93      0.85      1215
         low       0.91      0.75      0.82      1213

    accuracy                           0.84      2428
   macro avg       0.85      0.84      0.84      2428
weighted avg       0.85      0.84      0.84      2428

              precision    recall  f1-score   support

        high       0.30      0.67      0.41        60
         low       0.94      0.77      0.85       412

    accuracy                           0.76       472
   macro avg       0.62      0.72      0.63       472
weighted avg       0.86      0.76      0.79       472



array([[1124,   91],
       [ 300,  913]], dtype=int64)

In [118]:
confusion_matrix(stimYtest,vtcypred)

array([[ 40,  20],
       [ 94, 318]], dtype=int64)

In [119]:
#check classification report for both
vtcypred = polyvtc.predict(stimXtest)
vtcytrain = polyvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

0.9547384855089984 0.7923998896732869
              precision    recall  f1-score   support

        high       0.93      0.99      0.96      1215
         low       0.98      0.92      0.95      1213

    accuracy                           0.95      2428
   macro avg       0.96      0.95      0.95      2428
weighted avg       0.96      0.95      0.95      2428

              precision    recall  f1-score   support

        high       0.29      0.60      0.39        60
         low       0.93      0.78      0.85       412

    accuracy                           0.76       472
   macro avg       0.61      0.69      0.62       472
weighted avg       0.85      0.76      0.79       472



array([[1197,   18],
       [  92, 1121]], dtype=int64)

In [120]:
confusion_matrix(stimYtest,vtcypred)

array([[ 36,  24],
       [ 89, 323]], dtype=int64)

In [121]:
pkl.dump(stdvtc,open('finalstdvtc.pkl','wb'))
pkl.dump(polyvtc,open('finalpolyvtc.pkl','wb'))

In [122]:
adaboostvtc = pkl.load(open('adaoverallvtc.pkl','rb'))
adaboostvtc.fit(stimXtrain,stimYtrain)
vtcypred = adaboostvtc.predict(stimXtest)
vtcytrain = adaboostvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

1.0 0.8328288148928509
              precision    recall  f1-score   support

        high       1.00      1.00      1.00      1215
         low       1.00      1.00      1.00      1213

    accuracy                           1.00      2428
   macro avg       1.00      1.00      1.00      2428
weighted avg       1.00      1.00      1.00      2428

              precision    recall  f1-score   support

        high       0.34      0.37      0.35        60
         low       0.91      0.90      0.90       412

    accuracy                           0.83       472
   macro avg       0.63      0.63      0.63       472
weighted avg       0.84      0.83      0.83       472



array([[1215,    0],
       [   0, 1213]], dtype=int64)

In [123]:
finalvtc =VotingClassifier([('ada',adaboostvtc),('stdmodel',stdvtc),('polymodel',polyvtc)]
                           ,voting='soft',verbose = True,n_jobs=-1)






In [124]:

vtcypred = finalvtc.predict(stimXtest)
vtcytrain = finalvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
plot_confusion_matrix(finalvtc,stimXtest, stimYtest)

In [ ]:
pkl.dump(finalvtc,open('finaloverallvtc.pkl','wb'))

In [ ]:
stimXtrain

In [134]:
def fitvtc(Xtrain,Ytrain,Xtest,Ytest):
    vtc =VotingClassifier([('ada',adaboostvtc),('stdmodel',stdvtc),('polymodel',polyvtc)]
                           ,voting='soft',verbose = True,n_jobs=-1)

    vtc.fit(Xtrain,Ytrain)
    
    ypred = vtc.predict(Xtest)
    vtcytrain = vtc.predict(Xtrain)

    print(f1_score(Ytrain,vtcytrain,average='weighted'),f1_score(Ytest,ypred,average='weighted'))
    print(classification_report(Ytrain, vtcytrain))
    print(classification_report(Ytest, ypred))



    confusion_matrix(Ytrain, vtcytrain)
    return vtc

In [136]:
stimvtc=fitvtc(stimXtrain,stimYtrain,stimXtest,stimYtest)
depvtc=fitvtc(depXtrain,depYtrain,depXtest,depYtest)
haluvtc=fitvtc(haluXtrain,haluYtrain,haluXtest,haluYtest)

0.9472170308348915 0.8140304948867845
              precision    recall  f1-score   support

        high       0.92      0.98      0.95      1215
         low       0.98      0.91      0.95      1213

    accuracy                           0.95      2428
   macro avg       0.95      0.95      0.95      2428
weighted avg       0.95      0.95      0.95      2428

              precision    recall  f1-score   support

        high       0.32      0.58      0.41        60
         low       0.93      0.82      0.87       412

    accuracy                           0.79       472
   macro avg       0.63      0.70      0.64       472
weighted avg       0.85      0.79      0.81       472

0.9497672024723559 0.7252824858757062
              precision    recall  f1-score   support

        high       0.93      0.98      0.95      1177
         low       0.98      0.92      0.95      1174

    accuracy                           0.95      2351
   macro avg       0.95      0.95      0.95      235

In [138]:
pkl.dump(stimvtc,open('stimvtc.pkl','wb'))
pkl.dump(depvtc,open('depvtc.pkl','wb'))
pkl.dump(haluvtc,open('haluvtc.pkl','wb'))

In [ ]:
depvtc.predict(depXtrain)